In [5]:
import tensorflow as tf
import numpy as np
import pandas as pd

from tensorflow import keras

In [18]:
model = keras.models.load_model("models/TA94_7_26_23.h5")

In [19]:
def parse(input_string, target_rows=15):
    positions = input_string.split("%")

    chunk_df = pd.DataFrame([list(map(float, pos.split(","))) for pos in positions], columns=["x", "y", "z"])

    if len(chunk_df) == target_rows:
        return chunk_df

    linspace = pd.DataFrame(
        index=pd.RangeIndex(target_rows),
        columns=chunk_df.columns
    )

    for col in ["x", "y", "z"]:
        linspace[col] = pd.Series(np.interp(
            np.linspace(0, 1, num=target_rows),
            np.linspace(0, 1, num=len(chunk_df)),
            chunk_df[col]
        ))

    return linspace.values.tolist()

In [21]:
data_string = "4.5,2.98,-3.75%5.93,3,-3.46%7.36,3.13,-2.61%60.06,8,28.82%43.03,6.76,32.49"

custom_test_positions = parse(data_string)
custom_test_positions = np.array(custom_test_positions)

prediction = model.predict(custom_test_positions.reshape(-1, 15, 3))[0][0]
print(f"raw prediction: {prediction}")
print(f"rounded: {np.round(prediction)}")

1/1 [==============================] - 0s 21ms/step
raw prediction: 0.00025542176445014775
rounded: 0.0
